### ADD DEBUG 

In [ ]:
# --- COMPLETE CONTEXT AND TEST SCRIPT (v2 - ALL FUNCTIONS INCLUDED) ---
# This script contains the final refactored code and a self-contained test case.
# Running this single cell will execute the test.

import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint
import io

import pandas as pd
import numpy as np

from datetime import datetime, date
from IPython.display import display, Markdown
from pathlib import Path

# --- A. HELPER FUNCTIONS (Shared across tools) ---

def calculate_gain(price_series: pd.Series):
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

# --- B. THE CORE CALCULATION ENGINE, UPGRADED WITH DEBUG MODE (Headless, No UI) ---

def run_walk_forward_step_original_v0(df_close_full, df_high_full, df_low_full,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker):
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    safe_start_date = max(start_date, min_date_available)
    safe_calc_end_date = min(start_date + calc_period, max_date_available)
    safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
    if safe_start_date >= safe_calc_end_date: return {'error': "Invalid date range."}
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2: return {'error': "Not enough data in calc period."}

    metric_values = {}
    first_prices = calc_close.bfill().iloc[0]; last_prices = calc_close.ffill().iloc[-1]
    metric_values['Price'] = (last_prices / first_prices).dropna()
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean()
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display: return {'error': "No tickers found for the selected rank."}
        
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    
    return { 'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data, 'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series, 'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts, 'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()), 'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()), 'error': None }

def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker,
                          debug=False):  # <-- New debug argument
    """
    Core engine for a single walk-forward step.
    Returns a tuple: (results_dict, debug_data_dict).
    If debug=False, debug_data_dict will be None.
    """
    debug_data = {} if debug else None

    # --- Date Range Calculation (unchanged) ---
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    safe_start_date = max(start_date, min_date_available)
    safe_calc_end_date = min(start_date + calc_period, max_date_available)
    safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
    if safe_start_date >= safe_calc_end_date:
        return ({'error': "Invalid date range."}, None)
    
    # --- Metric Calculation (mostly unchanged, but we capture intermediates for debug) ---
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2:
        return ({'error': "Not enough data in calc period."}, None)

    # These are the components for our metrics
    first_prices = calc_close.bfill().iloc[0]
    last_prices = calc_close.ffill().iloc[-1]
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns = daily_returns.mean()
    std_returns = daily_returns.std()
    
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean()

    # We still build the main metric_values dict for ranking
    metric_values = {}
    metric_values['Price'] = (last_prices / first_prices).dropna()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

    # --- START: DEBUG DATA GENERATION ---
    if debug:
        # 1. 'ranking_metrics' DataFrame: The "Report Card"
        df_ranking = pd.DataFrame({
            'FirstPrice': first_prices,
            'LastPrice': last_prices,
            'MeanDailyReturn': mean_returns,
            'StdDevDailyReturn': std_returns,
            'MeanATRP': atrp,
            'Metric_Price': metric_values['Price'],
            'Metric_Sharpe': metric_values['Sharpe'],
            'Metric_SharpeATR': metric_values['Sharpe (ATR)']
        })
        df_ranking.index.name = 'Ticker'
        debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
    # --- END: DEBUG DATA GENERATION ---

    # --- Portfolio Selection (unchanged) ---
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display:
        return ({'error': "No tickers found for the selected rank."}, None)

    # --- Performance Calculation (unchanged) ---
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    # ... (rest of perf_data calculations are unchanged)
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)


    # --- Results DataFrame (unchanged) ---
    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])

    # --- START: DEBUG DATA GENERATION ---
    if debug:
        # 2. 'portfolio_trace' DataFrame: The "Daily Journal"
        df_trace = normalized_plot_data.copy()
        df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
        df_trace['Norm_Price_Portfolio'] = portfolio_series
        
        if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
            norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
            df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm

        # Add daily returns for easy inspection
        for col in df_trace.columns:
            if 'Norm_Price' in col:
                return_col_name = col.replace('Norm_Price', 'Return')
                df_trace[return_col_name] = df_trace[col].pct_change()
        
        debug_data['portfolio_trace'] = df_trace
    # --- END: DEBUG DATA GENERATION ---

    # --- Final Return Object (packaging results and debug data) ---
    final_results = {
        'tickers_to_display': tickers_to_display,
        'normalized_plot_data': normalized_plot_data,
        'portfolio_series': portfolio_series,
        'benchmark_price_series': benchmark_price_series,
        'performance_data': perf_data,
        'results_df': results_df,
        'actual_calc_end_ts': actual_calc_end_ts,
        'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()),
        'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()),
        'error': None
    }
    
    return (final_results, debug_data) # <-- Return a tuple


# --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS (get_eligible_universe CORRECTED) ---

def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
    """
    Calculates rolling quality metrics for OHLCV data to identify tradable tickers.

    This function enriches the input DataFrame with metrics that quantify data
    quality and liquidity over a specified rolling window.

    Args:
        df_ohlcv (pd.DataFrame): DataFrame with a ('Ticker', 'Date') MultiIndex
                                 and columns for OHLCV data.
        window (int): The lookback period in days for the rolling calculations.
                      Defaults to 252 (approx. one trading year).
        min_periods (int): The minimum number of observations in the window required
                           to have a value. Defaults to 126 (approx. half a year).
        debug (bool): If True, returns a DataFrame with all intermediate
                      calculations. Defaults to False, returning only the
                      final quality metrics.

    Returns:
        pd.DataFrame: A DataFrame containing the calculated quality metrics. See the
                      'Metrics Description' section below for column details. If debug is True,
                      the output will also include original and intermediate columns.

    Metrics Description:
        RollingStalePct (float):
            The rolling percentage (0.0 to 1.0) of days considered 'stale' within
            the lookback window. A day is flagged as stale if its trading volume is
            zero OR its high price is equal to its low price. This metric helps
            identify non-trading assets or assets with poor quality data feeds.
            **A lower value is better.**

        RollingMedianVolume (float):
            The rolling median of the daily dollar volume (Adj Close * Volume). The
            median is used to provide a robust measure of typical liquidity that is
            insensitive to single-day volume spikes (outliers). This metric is crucial
            for filtering out illiquid stocks where trades could incur significant
            slippage.
            **A higher value is better.**

        RollingSameVolCount (float):
            A rolling count of the number of times a day's volume was exactly equal
            to the previous day's volume. This is a heuristic for detecting
            potential low-quality data feeds, as this event is statistically rare
            for actively traded assets and may indicate improper forward-filling of
            missing data.
            **A lower value is better.**
    """
    print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
    df = df_ohlcv.copy()

    # Improvement 1: Ensure data is sorted for correctness
    if not df.index.is_monotonic_increasing:
        print("ℹ️ Data is not sorted. Sorting index chronologically...")
        df.sort_index(inplace=True)

    # --- Intermediate calculations ---
    # This calculation is always the same, whether in debug mode or not.
    df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
    df['DollarVolume'] = df['Adj Close'] * df['Volume']
    df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
    
    # === NEW: Add component columns ONLY if in debug mode ===
    if debug:
        print("...Adding debug component columns for 'IsStale'.")
        df['Debug_HasZeroVolume'] = (df['Volume'] == 0).astype(int)
        df['Debug_IsHighEqLow'] = (df['Adj High'] == df['Adj Low']).astype(int)
    # =========================================================

    # --- Rolling calculations ---
    grouped = df.groupby(level='Ticker')
    stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
    median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
    same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()

    quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
    quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
    quality_df.index = quality_df.index.droplevel(0)

    print("✅ Rolling metrics calculation complete.")
    
    if debug:
        # For debugging, return the original data joined with all calculations
        print("...Debug mode enabled, returning full calculation trace.")
        # The 'df' DataFrame now contains the extra debug columns, which will be included automatically.
        full_df = df.join(quality_df)
        return full_df
    else:
        # Default production behavior
        return quality_df

def get_eligible_universe_original(quality_metrics_df, filter_date, thresholds):
    """
    Filters tickers to create an eligible universe for a specific date based on quality metrics.

    Args:
        quality_metrics_df (pd.DataFrame): The output from calculate_rolling_quality_metrics.
        filter_date (pd.Timestamp or str): The specific date to perform the filtering on.
        thresholds (dict): A dictionary with the filtering rules, e.g.,
                           {'min_median_dollar_volume': 1e6, 'max_stale_pct': 0.05, 'max_same_vol_count': 1}.

    Returns:
        list: A list of ticker symbols that are eligible on the filter_date.
    """
    try:
        metrics_on_date = quality_metrics_df.xs(pd.to_datetime(filter_date), level='Date')
    except KeyError:
        print(f"Warning: Filter date {pd.to_datetime(filter_date).date()} not found in quality metrics index. Returning all tickers.")
        return quality_metrics_df.index.get_level_values('Ticker').unique().tolist()

    mask = (
        (metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
        (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
        (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
    )
    
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    
    print(f"Dynamic Filter ({pd.to_datetime(filter_date).date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers    

def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
    """
    Filters tickers to create an eligible universe for a specific date based on quality metrics.
    If the exact filter_date is not available, it uses the most recent previous date.
    This version is compatible with older pandas versions.
    """

    filter_date_ts = pd.to_datetime(filter_date)
    
    # Get the unique dates available in the index for efficient searching
    # Ensure they are sorted for the logic below to work correctly
    date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()

    # Handle edge case where the requested date is before any data exists
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point ({date_index[0].date()}). Returning empty universe.")
        return []

    # --- REVISED LOGIC FOR PANDAS COMPATIBILITY ---
    # Find all dates that are less than or equal to the requested filter date
    valid_prior_dates = date_index[date_index <= filter_date_ts]

    if valid_prior_dates.empty:
        # This case should be rare given the edge case check above, but it's good practice
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []

    # The date to use is the last one in this sorted list (the most recent one)
    actual_date_to_use = valid_prior_dates[-1]
    # --- END OF REVISED LOGIC ---

    # If we had to fall back to a previous date, inform the user
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
        
    # Now, we are guaranteed to have a valid date to select with .xs()
    metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')

    mask = (
        (metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
        (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
        (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
    )
    
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    
    # We use the *original* requested date in the message for clarity.
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers



# --- D. THE UI WRAPPER UPGRADED WITH DEBUG MODE ---

def plot_walk_forward_analyzer_original_v0(df_ohlcv, 
                               default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO'):
    print("Initializing Walk-Forward Analyzer...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

        results = run_walk_forward_step_original_v0(df_close_full, df_high_full, df_low_full, start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker)
        
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}")
            return
            
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]
                    trace.x, trace.y, trace.name = results['normalized_plot_data'].index, results['normalized_plot_data'][ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results['results_df']
        
        with ticker_list_output:
            print(f"Analyzing from {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            print(f"  - Ranking based on performance from {results['safe_start_date'].date()} to {results['actual_calc_end_ts'].date()}.")
            pprint.pprint(results['tickers_to_display'], width=120, compact=True)
            
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container

def plot_walk_forward_analyzer_v1(df_ohlcv, 
                               default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO',
                               # NEW: Add quality thresholds as an argument
                               quality_thresholds={'min_median_dollar_volume': 10_000_000, # $10 million median daily trade volume
                                                   'max_stale_pct': 0.1,                   # Allow 10% stale days (i.e. Volume=0 or High=Low)
                                                   'max_same_vol_count': 1                 # Allow at most 1 suspicious volume event (i.e. same Volume on consecutive day)
                                                  }):
    
    print("Initializing Walk-Forward Analyzer (with Dynamic Universe Filtering)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    
    # --- MODIFICATION 1: PRE-CALCULATE QUALITY METRICS (Mirrors the bot) ---
    print("Pre-calculating data quality metrics...")
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    # --------------------------------------------------------------------
    
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

        # --- MODIFICATION 2: DYNAMIC UNIVERSE SELECTION (Mirrors the bot) ---
        eligible_tickers = get_eligible_universe(
            quality_metrics_df,
            filter_date=start_date,
            thresholds=quality_thresholds # Use the new argument
        )

        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {start_date.date()} with the current quality filters.")
            return
            
        # Filter the main dataframes to only include eligible tickers for this run
        df_close_step = df_close_full[eligible_tickers]
        df_high_step = df_high_full[eligible_tickers]
        df_low_step = df_low_full[eligible_tickers]
        # ---------------------------------------------------------------------

        # --- MODIFICATION 3: PASS THE FILTERED DATA (Mirrors the bot) ---
        results = run_walk_forward_step_original_v0(
            df_close_step, df_high_step, df_low_step, # Use the filtered dataframes
            start_date, calc_period, fwd_period, 
            metric, rank_start, rank_end, benchmark_ticker
        )
        # ---------------------------------------------------------------------
        
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}")
            return
            
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]
                    # IMPORTANT: Plotting uses the original full dataframe to show the price history
                    # even if the ticker wasn't in the universe for the whole period.
                    plot_data_series = df_close_full[ticker].loc[results['safe_start_date']:results['safe_viz_end_date']]
                    normalized_series = plot_data_series / plot_data_series.bfill().iloc[0]
                    trace.x, trace.y, trace.name = normalized_series.index, normalized_series.values, ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
            else: benchmark_trace.visible = False
            
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results['results_df']
        
        with ticker_list_output:
            # (The rest of the display logic is unchanged)
            print(f"Analyzing from {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            print(f"  - Ranking based on performance from {results['safe_start_date'].date()} to {results['actual_calc_end_ts'].date()}.")
            pprint.pprint(results['tickers_to_display'], width=120, compact=True)
            
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 10_000_000, # $10 million median daily trade volume
                                                   'max_stale_pct': 0.1,                   # Allow 10% stale days (i.e. Volume=0 or High=Low)
                                                   'max_same_vol_count': 1                 # Allow at most 1 suspicious volume event (i.e. same Volume on consecutive day)
                                                  },
                               debug=False): # <-- New debug argument
    
    print("Initializing Walk-Forward Analyzer (with Dynamic Universe Filtering)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    
    print("Pre-calculating data quality metrics...")
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    # --- UI Widgets (unchanged) ---
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    
    results_container = [None]
    debug_data_container = [None] # <-- New container for debug data

    # --- Plotly Figure (unchanged) ---
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

        eligible_tickers = get_eligible_universe(
            quality_metrics_df, filter_date=start_date, thresholds=quality_thresholds
        )
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {start_date.date()} with the current quality filters.")
            return
            
        df_close_step = df_close_full[eligible_tickers]
        df_high_step = df_high_full[eligible_tickers]
        df_low_step = df_low_full[eligible_tickers]

        # --- MODIFIED: Capture both results and debug data ---
        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step,
            start_date, calc_period, fwd_period, 
            metric, rank_start, rank_end, benchmark_ticker,
            debug=debug # <-- Pass the debug flag through
        )
        
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}")
            return

        # --- Update plot (mostly unchanged) ---
        with fig.batch_update():
            # (Plotting logic remains the same)
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]
                    plot_data_series = df_close_full[ticker].loc[results['safe_start_date']:results['safe_viz_end_date']]
                    normalized_series = plot_data_series / plot_data_series.bfill().iloc[0]
                    trace.x, trace.y, trace.name = normalized_series.index, normalized_series.values, ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        # --- Store results and debug data in their containers ---
        results_container[0] = results['results_df']
        debug_data_container[0] = debug_output
        
        # --- Print summary (unchanged) ---
        with ticker_list_output:
            print(f"Analyzing from {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            print(f"  - Ranking based on performance from {results['safe_start_date'].date()} to {results['actual_calc_end_ts'].date()}.")
            pprint.pprint(results['tickers_to_display'], width=120, compact=True)
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    
    return (results_container, debug_data_container) # <-- Return a tuple

# --- E. VERIFICATION TOOLS ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period, export_csv=False):
    # This function is unchanged
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`"))
    display(Markdown(f"**Benchmark Ticker:** `{benchmark_ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts_theoretical = start_date_ts + calc_offset
    fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
    actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Calculation Period End Date:** `{actual_calc_end_ts.date()}`"))
    display(Markdown(f"**Forward Period End Date:** `{fwd_end_date_ts_theoretical.date()}`"))
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
    normalized_portfolio_prices = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0])
    portfolio_value_series = normalized_portfolio_prices.mean(axis=1)
    try: benchmark_price_series = df_close_full[benchmark_ticker]
    except KeyError as e: print(f"---! ERROR: Ticker {e} not found !---"); return

    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price = price_series.bfill().iloc[0]; end_price = price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"  - Start Value (on {price_series.first_valid_index().date()}): {start_price:,.4f}\n  - End Value   (on {price_series.last_valid_index().date()}): {end_price:,.4f}\n  - Gain = ({end_price:,.4f} / {start_price:,.4f}) - 1 = {gain:.2%}")
        returns = price_series.pct_change()
        mean_return = returns.mean(); std_return = returns.std()
        sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 and std_return != np.inf else np.nan
        print(f"\n  - Mean Daily Return: {mean_return:.6f}\n  - Std Dev of Daily Return: {std_return:.6f}\n  - Sharpe = ({mean_return:.6f} / {std_return:.6f}) * sqrt(252) = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    display(Markdown("### A. Calculation Period Analysis ('In-Sample')"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
    perf_calc_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
    display(Markdown("\n### B. Forward Period Analysis ('Moment of Truth')"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    perf_fwd_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### C. Full Period Analysis (Total)"))
    perf_full_p = print_verification_steps("Group Portfolio", portfolio_value_series)
    perf_full_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### D. Final Summary Table (matches analyzer output)"))
    rows = []
    rows.append({'Metric': 'Group Portfolio Gain', 'Full': perf_full_p['gain'], 'Calc': perf_calc_p['gain'], 'Fwd': perf_fwd_p['gain']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': perf_full_b['gain'], 'Calc': perf_calc_b['gain'], 'Fwd': perf_fwd_b['gain']})
    rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': perf_full_p['gain'] - perf_full_b['gain'], 'Calc': perf_calc_p['gain'] - perf_calc_b['gain'], 'Fwd': perf_fwd_p['gain'] - perf_fwd_b['gain']})
    rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': perf_full_p['sharpe'], 'Calc': perf_calc_p['sharpe'], 'Fwd': perf_fwd_p['sharpe']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': perf_full_b['sharpe'], 'Calc': perf_calc_b['sharpe'], 'Fwd': perf_fwd_b['sharpe']})
    rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': perf_full_p['sharpe'] - perf_full_b['sharpe'], 'Calc': perf_calc_p['sharpe'] - perf_calc_b['sharpe'], 'Fwd': perf_fwd_p['sharpe'] - perf_fwd_b['sharpe']})
    report_df = pd.DataFrame(rows).set_index('Metric')
    gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
    sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
    styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
    display(styled_df)
    if export_csv:
        export_df = pd.DataFrame({'Portfolio_Value_Normalized': portfolio_value_series, 'Portfolio_Return': portfolio_value_series.pct_change(), f'Benchmark_Price_{benchmark_ticker}': benchmark_price_series})
        filename = f"verification_group_tickers_{start_date_ts.strftime('%Y%m%d')}.csv"
        export_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed group verification data exported to '{filename}'")

def verify_ticker_ranking_metrics(df_ohlcv, 
                                  ticker, 
                                  start_date, 
                                  calc_period, 
                                  fwd_period, 
                                  export_csv=False):
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    try: df_ticker = df_ohlcv.loc[ticker].sort_index()
    except KeyError: print(f"---! ERROR: Ticker '{ticker}' not found !---"); return
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Requested Calculation Period:** `{start_date_ts.date()}` to `{calc_end_date_ts.date()}`"))
    display(Markdown(f"**Requested Forward Period:**   `{calc_end_date_ts.date()}` to `{fwd_end_date_ts.date()}`"))
    display(Markdown("### A. Calculation Period Analysis (for Ranking Metrics)"))
    calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
    if calc_df['Adj Close'].notna().sum() < 2: print("\n---! ERROR: Not enough data points !---"); return
    actual_calc_end_date = calc_df.index.max().date()
    display(Markdown(f"**Actual Dates Used:** `{calc_df.index.min().date()}` to `{actual_calc_end_date}`"))
    calc_gain = calculate_gain(calc_df['Adj Close'])
    calc_start_price = calc_df['Adj Close'].bfill().iloc[0]
    calc_end_price = calc_df['Adj Close'].ffill().iloc[-1]
    display(Markdown("#### `CalcGain` Verification:"))
    print(f"  - Calc Start Price: ${calc_start_price:.2f}\n  - Calc End Price:   ${calc_end_price:.2f}  <-- 'CalcPrice'\n  - CalcGain = {calc_gain:.2%}")
    display(Markdown("#### `MetricValue` Verification:"))
    price_metric = (calc_end_price / calc_start_price)
    print(f"\n1. Price Metric:\n   - Formula: Last Price / First Price = {price_metric:.4f}")
    daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"\n2. Sharpe Metric:\n   - Mean Daily Return: {daily_returns.mean():.6f}\n   - Std Dev Daily Return: {daily_returns.std():.6f}\n   - Annualized Sharpe = {sharpe_ratio:.4f}")
    print(f"\n3. Sharpe (ATR) Metric:")
    tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp_series = atr / calc_df['Adj Close']
    atrp_mean = atrp_series.mean()
    sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
    print(f"   - Mean Daily Return: {daily_returns.mean():.6f} (same as above)\n   - Average ATR Percent (ATRP): {atrp_mean:.6f}\n   - Sharpe (ATR) = {sharpe_atr:.4f}")
    display(Markdown("\n### B. Forward Period Analysis (`FwdGain`)"))
    fwd_df = df_ticker.loc[actual_calc_end_date:fwd_end_date_ts].copy()
    fwd_gain = calculate_gain(fwd_df['Adj Close'])
    fwd_end_price = fwd_df['Adj Close'].ffill().iloc[-1] if fwd_gain is not np.nan else calc_end_price
    print(f"  - Fwd Start Price (Calc End Price): ${calc_end_price:.2f}\n  - Fwd End Price: ${fwd_end_price:.2f}\n  - FwdGain = {fwd_gain:.2%}")
    display(Markdown("\n### C. Final Summary Table"))
    summary_data = {'Metric': ['Price', 'Sharpe', 'Sharpe (ATR)'], 'Calculated Value': [f"{price_metric:.4f}", f"{sharpe_ratio:.4f}", f"{sharpe_atr:.4f}"], 'Corresponds To': ['`MetricValue`', '`MetricValue`', '`MetricValue`'], '---': ['---','---','---'], 'Gain Metric': ['Calc Period Gain', 'Forward Period Gain'], 'Gain Value': [f"{calc_gain:.2%}", f"{fwd_gain:.2%}"], 'Gain Corresponds To': ['`CalcGain`', '`FwdGain`']}
    summary_df = pd.DataFrame(summary_data)
    display(summary_df.style.hide(axis="index"))
    
    if export_csv:
        calc_df['Period'] = 'Calculation'; calc_df['Daily_Return'] = daily_returns; calc_df['True_Range'] = tr; calc_df['ATR_14'] = atr; calc_df['ATRP'] = atrp_series
        fwd_df['Period'] = 'Forward'
        combined_df = pd.concat([calc_df, fwd_df.iloc[1:]])
        filename = f"verification_ticker_{ticker}_{start_date_ts.strftime('%Y%m%d')}.csv"
        combined_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed ticker data exported to '{filename}'")


# # --- F. DATA SOURCE: df_OHLCV.info() ---
# '''
# <class 'pandas.core.frame.DataFrame'>
# MultiIndex: 2115382 entries, ('A', Timestamp('2020-01-02 00:00:00')) to ('ZWS', Timestamp('2025-10-03 00:00:00'))
# Data columns (total 5 columns):
#  #   Column     Dtype  
# ---  ------     -----  
#  0   Adj Open   float64
#  1   Adj High   float64
#  2   Adj Low    float64
#  3   Adj Close  float64
#  4   Volume     int64  
# dtypes: float64(4), int64(1)
# memory usage: 88.9+ MB
# '''


# # --- 1. SAMPLE DATA (for reproducibility) ---
# date_rng = pd.date_range(start='2023-01-01', end='2023-10-31', freq='B')
# tickers = ['CVNA', 'VRT', 'APP', 'SMCI', 'IONQ', 'XPO', 'XP', 'AD', 'USM', 'MOD', 'VOO', 'QQQ']
# data = []
# np.random.seed(42) # for reproducibility
# for ticker in tickers:
#     price = 100 + (np.random.randn(len(date_rng)).cumsum() * (0.5 if ticker != 'SMCI' else 2.5))
#     high = price + np.random.uniform(0, 2, size=len(date_rng))
#     low = price - np.random.uniform(0, 2, size=len(date_rng))
#     open_price = price + np.random.uniform(-1, 1, size=len(date_rng))
#     volume = np.random.randint(100000, 5000000, size=len(date_rng))
#     ticker_df = pd.DataFrame({'Date': date_rng,'Ticker': ticker,'Adj Open': open_price,'Adj High': high,'Adj Low': low,'Adj Close': price,'Volume': volume})
#     data.append(ticker_df)
# df_full = pd.concat(data)
# df_OHLCV_test = df_full.set_index(['Ticker', 'Date'])

# # --- 2. TEST EXECUTION ---
# test_start_date = '2023-04-01'
# test_calc_period = '6M'
# test_fwd_period = '2W'
# test_metric = 'Price'
# test_rank_start = 1
# test_rank_end = 10
# test_benchmark = 'VOO'

# print("--- RUNNING REFACTORED CODE TEST ---")
# walk_forward_results = plot_walk_forward_analyzer_original_v0(
#     df_OHLCV_test,
#     default_start_date=test_start_date,
#     default_calc_period=test_calc_period,
#     default_fwd_period=test_fwd_period,
#     default_metric=test_metric,
#     default_rank_start=test_rank_start,
#     default_rank_end=test_rank_end,
#     default_benchmark_ticker=test_benchmark
# )
# print("\n--- TEST COMPLETE ---")

In [ ]:
download_path = Path.home() / "Downloads"  
# OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')
print(f'df_OHLCV.info() :\n{df_OHLCV.info()}')
print(f'\ndf_OHLCV.head():\n{df_OHLCV.head()}')
print(f'\ndf_OHLCV.tail():\n{df_OHLCV.tail()}')

In [ ]:
plot_walk_forward_analyzer_original_v0(
    df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
    default_start_date='2017-04-30',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO'
)

In [ ]:
plot_walk_forward_analyzer_v1(
    df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
    default_start_date='2017-04-30',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO'
)

In [ ]:
plot_walk_forward_analyzer(
    df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
    default_start_date='2017-04-30',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO'
)

In [ ]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_dev,
    default_start_date='2014-07-31',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO',
    quality_thresholds=bot_config['quality_thresholds'],
    debug=True  # <-- Activate the new mode!
)

#######################################

In [ ]:
# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS (get_eligible_universe MODIFIED) ---

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     """
#     Filters tickers to create an eligible universe for a specific date based on quality metrics.
#     If the exact filter_date is not available, it uses the most recent previous date.
#     """
#     filter_date_ts = pd.to_datetime(filter_date)
    
#     # Get the unique dates available in the index for efficient searching
#     date_index = quality_metrics_df.index.get_level_values('Date').unique()

#     # Handle edge case where the requested date is before any data exists
#     if filter_date_ts < date_index.min():
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []

#     try:
#         # Find the integer position of the last valid date that is <= filter_date
#         # This is the key to finding the previous available date robustly.
#         loc = date_index.get_loc(filter_date_ts, method='ffill')
#         actual_date_to_use = date_index[loc]
#     except Exception as e:
#         print(f"Error finding a valid date for {filter_date_ts.date()}: {e}. Returning empty universe.")
#         return []

#     # If we had to fall back to a previous date, inform the user (as an info message, not a warning)
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
        
#     # Now, we are guaranteed to have a valid date to select with .xs()
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')

#     mask = (
#         (metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#         (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#         (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
#     )
    
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
    
#     # The original print message is still very useful.
#     # We use the *original* requested date in the message for clarity.
#     print(f"Dynamic Filter ({pd.to_datetime(filter_date).date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers



In [ ]:
# --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS (get_eligible_universe CORRECTED) ---

def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
    """
    Filters tickers to create an eligible universe for a specific date based on quality metrics.
    If the exact filter_date is not available, it uses the most recent previous date.
    This version is compatible with older pandas versions.
    """

    filter_date_ts = pd.to_datetime(filter_date)
    
    # Get the unique dates available in the index for efficient searching
    # Ensure they are sorted for the logic below to work correctly
    date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()

    # Handle edge case where the requested date is before any data exists
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point ({date_index[0].date()}). Returning empty universe.")
        return []

    # --- REVISED LOGIC FOR PANDAS COMPATIBILITY ---
    # Find all dates that are less than or equal to the requested filter date
    valid_prior_dates = date_index[date_index <= filter_date_ts]

    if valid_prior_dates.empty:
        # This case should be rare given the edge case check above, but it's good practice
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []

    # The date to use is the last one in this sorted list (the most recent one)
    actual_date_to_use = valid_prior_dates[-1]
    # --- END OF REVISED LOGIC ---

    # If we had to fall back to a previous date, inform the user
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
        
    # Now, we are guaranteed to have a valid date to select with .xs()
    metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')

    mask = (
        (metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
        (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
        (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
    )
    
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    
    # We use the *original* requested date in the message for clarity.
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers


In [ ]:
import pandas as pd
import numpy as np

# # --- 1. SETUP: Create a sample df_OHLCV (Replace with your data loading) ---
# # This part simulates your full dataset. In your final script, you would
# # load your actual data here.
# print("--- Creating a sample DataFrame to demonstrate the split... ---")
# tickers = ['AAPL', 'MSFT', 'GOOG']
# date_rng = pd.date_range(start='1999-11-18', end='2025-10-03', freq='B')
# multi_idx = pd.MultiIndex.from_product([tickers, date_rng], names=['Ticker', 'Date'])

# # Create a dummy DataFrame with the correct structure
# df_OHLCV = pd.DataFrame(
#     np.random.rand(len(multi_idx), 5),
#     index=multi_idx,
#     columns=['Adj Open', 'Adj High', 'Adj Low', 'Adj Close', 'Volume']
# )
# print("Sample DataFrame created successfully.\n")
# # --- End of Setup Section ---


# --- 2. DEFINE THE SPLIT DATE ---
# This is the last day of our In-Sample (IS) "Discovery Zone".
split_date = pd.to_datetime('2018-12-31')

print(f"Splitting data on: {split_date.date()}")
print("="*40)


# --- 3. PERFORM THE SPLIT ---
# We access the 'Date' level of the MultiIndex to create our boolean masks.

# In-Sample (IS) DataFrame: Data for discovery and training the bot.
df_IS = df_OHLCV[df_OHLCV.index.get_level_values('Date') <= split_date].copy()

# Out-of-Sample (OOS) DataFrame: Data held back for final validation.
df_OOS = df_OHLCV[df_OHLCV.index.get_level_values('Date') > split_date].copy()

# Using .copy() is good practice to avoid SettingWithCopyWarning later on.


# --- 4. VERIFY THE SPLIT ---
# Always check your work to ensure the split was done correctly.

print("\n--- In-Sample (IS) 'Discovery Zone' Info ---")
df_IS.info(verbose=False, memory_usage='deep') # Use verbose=False for a cleaner summary
print(f"IS Date Range: {df_IS.index.get_level_values('Date').min().date()} to {df_IS.index.get_level_values('Date').max().date()}")
print(f"IS Shape: {df_IS.shape}")
print(f"Percentage of IS data: {df_IS.shape[0] / df_OHLCV.shape[0]:.2%}")

print("\n--- Out-of-Sample (OOS) 'Validation Zone' Info ---")
df_OOS.info(verbose=False, memory_usage='deep')
print(f"OOS Date Range: {df_OOS.index.get_level_values('Date').min().date()} to {df_OOS.index.get_level_values('Date').max().date()}")
print(f"OOS Shape: {df_OOS.shape}")
print(f"Percentage of OOS data: {df_OOS.shape[0] / df_OHLCV.shape[0]:.2%}")

# Final check
total_rows = df_IS.shape[0] + df_OOS.shape[0]
print(f"\nVerification: {df_IS.shape[0]} (IS) + {df_OOS.shape[0]} (OOS) = {total_rows} rows.")
print(f"Original total rows: {df_OHLCV.shape[0]} rows. Match: {total_rows == df_OHLCV.shape[0]}")

In [ ]:
# This code REPLACES the previous df_dev creation snippet.
# It should still be placed after df_IS and df_OOS are created.

# --- 5. (CORRECTED) CREATE A SMALLER "DEVELOPMENT SANDBOX" DATAFRAME ---
# We use a RECENT 5-year slice of our In-Sample data for rapid development.
# This is much faster and more representative of modern data.

dev_start_date = pd.to_datetime('2014-01-01')
dev_end_date = pd.to_datetime('2018-12-31') # This is the end of our df_IS period

print("\n--- Creating Development Sandbox DataFrame (Corrected) ---")
print(f"Slicing df_IS from {dev_start_date.date()} to {dev_end_date.date()}")
print("="*60)

# Create the development dataframe by slicing the main In-Sample data
df_dev = df_IS[(df_IS.index.get_level_values('Date') >= dev_start_date) &
               (df_IS.index.get_level_values('Date') <= dev_end_date)].copy()


# --- 6. VERIFY THE (CORRECTED) DEVELOPMENT DATAFRAME ---
print("\n--- Development Sandbox (df_dev) Info ---")
df_dev.info(verbose=False, memory_usage='deep')
print(f"df_dev Date Range: {df_dev.index.get_level_values('Date').min().date()} to {df_dev.index.get_level_values('Date').max().date()}")
print(f"df_dev Shape: {df_dev.shape}")
print(f"df_dev as percentage of IS data: {df_dev.shape[0] / df_IS.shape[0]:.2%}")

# --- Get unique tickers from the 'Ticker' level of the MultiIndex ---

# Get the 'Ticker' level of the index
ticker_index = df_dev.index.get_level_values('Ticker')

# Get the unique values from that level
unique_tickers = ticker_index.unique()

# Print the results
print("="*60)
print(f"\nFound {len(unique_tickers)} unique tickers in df_dev.")
print("First 10 unique tickers:")
print(unique_tickers[:10].tolist()) # .tolist() gives a cleaner printout for a slice
print("\nLast 10 unique tickers:")
print(unique_tickers[-10:].tolist())

In [ ]:
# --- 7. BOT CONFIGURATION MANAGER ---
# This dictionary defines the entire search space for our bot.
# We start with a small set of parameters for our development run on df_dev.

# bot_config = {
#     # --- Time Parameters ---
#     # Defines the overall period the bot will step through.
#     # For dev, we'll use the full range of df_dev.
#     'search_start_date': '2014-01-01',
#     'search_end_date': '2018-12-31',
#     'step_frequency': '3M',  # How often to re-rank and form a new portfolio. '1M', '3M', '6M' etc.

#     # --- Strategy Parameters (The Search Grid) ---
#     # The bot will test every possible combination of these lists.
#     'calc_periods': ['6M', '1Y'],
#     'fwd_periods': ['3M'], # Must match step_frequency for a non-overlapping backtest
#     'metrics': ['Sharpe', 'Sharpe (ATR)'],
#     'rank_slices': [
#         (1, 10),      # Top 10 stocks
#         (11, 30),     # Stocks ranked 11th through 30th
#     ],

#     # --- Data Quality Filter Parameters ---
#     # These thresholds will be used to create the eligible universe at each step.
#     'quality_thresholds': {
#         'min_median_dollar_volume': 1_000_000,
#         'max_stale_pct': 0.05,
#         'max_same_vol_count': 10
#     },

#     # --- General Parameters ---
#     'benchmark_ticker': 'VOO',
#     'results_output_path': './dev_strategy_search_results.csv'
# }


# --- 7. BOT CONFIGURATION MANAGER (UPDATED) ---

bot_config = {
    # --- Time Parameters ---
    'search_start_date': '2014-01-01',
    'search_end_date': '2018-12-31',
    # MODIFICATION: Changed '3M' to '3ME' for Month-End frequency to resolve deprecation warning.
    'step_frequency': '3ME',

    # --- Strategy Parameters (The Search Grid) ---
    'calc_periods': ['6M', '1Y'],
    'fwd_periods': ['3M'], 
    'metrics': ['Sharpe', 'Sharpe (ATR)'],
    'rank_slices': [
        (1, 10),
        (11, 30),
    ],

    # --- Data Quality Filter Parameters ---
    'quality_thresholds': {
        'min_median_dollar_volume': 10_000_000,
        'max_stale_pct': 0.1,
        'max_same_vol_count': 1,
    },

    # --- General Parameters ---
    'benchmark_ticker': 'VOO',
    'results_output_path': './export_csv/dev_strategy_search_results.csv'
}


# --- Let's quickly calculate how many simulations this configuration will run ---
from itertools import product

num_param_sets = len(list(product(
    bot_config['calc_periods'],
    bot_config['fwd_periods'],
    bot_config['metrics'],
    bot_config['rank_slices']
)))

# Estimate the number of time steps
time_steps = pd.date_range(
    start=bot_config['search_start_date'],
    end=bot_config['search_end_date'],
    freq=bot_config['step_frequency']
)

print("\n--- Bot Configuration Initialized ---")
print(f"Number of unique parameter combinations: {num_param_sets}")
print(f"Estimated number of time steps: {len(time_steps)}")
print(f"Total simulations to run: {num_param_sets * len(time_steps)}")

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from tqdm.auto import tqdm
import time
from IPython.display import display

# --- Assume all functions from our project context are already defined ---
# (calculate_rolling_quality_metrics, get_eligible_universe, run_walk_forward_step, etc.)

# --- Assume df_dev and bot_config are already defined from previous steps ---

def run_strategy_search(df_ohlcv, config):
    """
    Runs the main backtesting loop based on a provided configuration dictionary.
    """
    start_time = time.time()
    
    # --- 1. PRE-PROCESSING (Run once for efficiency) ---
    print("--- Phase 1: Pre-processing Data ---")
    
    # Calculate quality metrics for the entire development period
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)

    # Unstack the data for fast slicing later. This is a major optimization.
    print("Unstacking data for performance...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    print("✅ Pre-processing complete.\n")
    
    # --- 2. SETUP THE MAIN LOOP ---
    print("--- Phase 2: Setting up Simulation Loops ---")
    
    # Create all parameter combinations to test
    param_combinations = list(product(
        config['calc_periods'],
        config['fwd_periods'],
        config['metrics'],
        config['rank_slices']
    ))
    
    # Create the list of dates where the bot will re-evaluate the strategy
    step_dates = pd.date_range(
        start=config['search_start_date'],
        end=config['search_end_date'],
        freq=config['step_frequency']
    )
    
    # Map string periods to pandas DateOffset objects for our core function
    period_options = {
        '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3),
        '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)
    }
    
    results_log = []
    total_sims = len(param_combinations) * len(step_dates)
    print(f"Found {len(param_combinations)} parameter sets and {len(step_dates)} time steps.")
    print(f"Total simulations to run: {total_sims}")
    print("✅ Setup complete. Starting main loop...\n")

    # --- 3. RUN THE MAIN LOOP ---
    print("--- Phase 3: Running Simulations ---")
    
    # Use tqdm for a progress bar on the outer loop
    pbar = tqdm(param_combinations, desc="Parameter Sets")
    for params in pbar:
        # Unpack parameters for this run
        calc_period_str, fwd_period_str, metric, rank_slice = params
        calc_period = period_options[calc_period_str]
        fwd_period = period_options[fwd_period_str]
        rank_start, rank_end = rank_slice

        # Inner loop for stepping through time
        for step_date in step_dates:
            # 3a. DYNAMIC UNIVERSE SELECTION
            eligible_tickers = get_eligible_universe(
                quality_metrics_df,
                filter_date=step_date,
                thresholds=config['quality_thresholds']
            )

            if not eligible_tickers:
                # print(f"Warning: No eligible tickers on {step_date.date()}. Skipping.")
                continue
            
            # 3b. FILTER DATA FOR THIS STEP
            df_close_step = df_close_full[eligible_tickers]
            df_high_step = df_high_full[eligible_tickers]
            df_low_step = df_low_full[eligible_tickers]

            # 3c. RUN THE CORE ANALYSIS
            step_result = run_walk_forward_step(
                df_close_full=df_close_step,
                df_high_full=df_high_step,
                df_low_full=df_low_step,
                start_date=step_date,
                calc_period=calc_period,
                fwd_period=fwd_period,
                metric=metric,
                rank_start=rank_start,
                rank_end=rank_end,
                benchmark_ticker=config['benchmark_ticker']
            )
            
            # 3d. LOG THE RESULTS
            if step_result['error'] is None:
                p = step_result['performance_data']
                log_entry = {
                    'step_date': step_date.date(),
                    'calc_period': calc_period_str,
                    'fwd_period': fwd_period_str,
                    'metric': metric,
                    'rank_start': rank_start,
                    'rank_end': rank_end,
                    'num_universe': len(eligible_tickers),
                    'num_portfolio': len(step_result['tickers_to_display']),
                    'fwd_p_gain': p['fwd_p_gain'],
                    'fwd_b_gain': p['fwd_b_gain'],
                    'fwd_gain_delta': p['fwd_p_gain'] - p['fwd_b_gain'] if not np.isnan(p['fwd_b_gain']) else np.nan,
                    'fwd_p_sharpe': p['fwd_p_sharpe'],
                }
                results_log.append(log_entry)

    print("✅ Main loop finished.\n")

    # --- 4. SAVE THE RESULTS ---
    print("--- Phase 4: Saving Results ---")
    if not results_log:
        print("Warning: No results were generated. The output file will be empty.")
        return None

    final_df = pd.DataFrame(results_log)
    output_path = config['results_output_path']
    final_df.to_csv(output_path, index=False, float_format='%.4f')

    end_time = time.time()
    print(f"✅ Results saved to '{output_path}'")
    print(f"Total execution time: {end_time - start_time:.2f} seconds.")
    
    return final_df

# --- Execute the Bot ---
dev_results_df = run_strategy_search(df_dev, bot_config)

# --- Display a sample of the results ---
if dev_results_df is not None:
    print("\n--- Sample of Generated Results ---")
    display(dev_results_df.head())

In [ ]:
dev_results_df.to_csv('./export_csv/dev_results_df.csv')

In [ ]:
# Our inspection rules inside the rolling window
ticker_thresholds = {
    'min_median_dollar_volume': 10_000_000, # $10 million median daily trade volume
    'max_stale_pct': 0.1,                   # Allow 10% stale days (i.e. Volume=0 or High=Low)
    'max_same_vol_count': 1                 # Allow at most 1 suspicious volume event (i.e. same Volume on consecutive day)
}

In [ ]:
# --- Verification for the first row of bot results ---

print("--- Replicating the scenario from the first CSV row ---")
print("Start Date: 2014-07-31")
print("Calc Period: 6M, Fwd Period: 3M")
print("Metric: Sharpe, Ranks: 1 to 10")
print("--------------------------------------------------")
print("\nInstructions: The UI will load with the correct defaults. Simply click the 'Update Chart' button.")

# Call the plotter using the parameters from the CSV row as defaults
plot_walk_forward_analyzer(
    df_ohlcv=df_dev,  # CRITICAL: Use the same df_dev as the bot
    default_start_date='2017-04-30',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO',
    quality_thresholds=ticker_thresholds,
)

### Compare bot results vs verified plot_walk_forward_analyzer_original

In [ ]:
# pick the row
row_index = 43
row = dev_results_df.iloc[row_index]

# convert to dict and reformat step_date
row_dict = row.to_dict()
row_dict['step_date'] = row_dict['step_date'].strftime('%Y-%m-%d')
print(f'row no: {row_index}')
print(f'row_dict: {row_dict}')

_start_date=row_dict['step_date']
_calc_period=row_dict['calc_period']
_fwd_period=row_dict['fwd_period']
_metric=row_dict['metric']
_rank_start=row_dict['rank_start']
_rank_end=row_dict['rank_end']
_benchmark_ticker='VOO'

In [ ]:
plot_walk_forward_analyzer(
    df_ohlcv=df_dev,  # CRITICAL: Use the same df_dev as the bot
    default_start_date=_start_date,
    default_calc_period=_calc_period,
    default_fwd_period=_fwd_period,
    default_metric=_metric,
    default_rank_start=_rank_start,
    default_rank_end=_rank_end,
    default_benchmark_ticker=_benchmark_ticker,
)

In [ ]:
quality_df = calculate_rolling_quality_metrics(
    df_ohlcv=df_OHLCV,
    window=252,
    min_periods=126,
    debug=False,  # <-- The key to our new, improved workflow    
)

In [ ]:
# 1. make sure the whole frame is sorted
df_tmp = quality_df.sort_index()

# 2a. pick the exact date if it exists, otherwise the previous one
date_needed = pd.Timestamp(_start_date)
dates = df_tmp.index.get_level_values('Date').unique().sort_values()   # ← sorted
# first date >= date_needed  (later)
pos = dates.searchsorted(date_needed, side='left')
if pos == len(dates):          # date_needed is after the last date
    later_date = pd.NaT
else:
    later_date = dates[pos]

print(f'_start_date: {_start_date}')
print(f'real start date: {later_date}')

# 3. slice
# quality_df = df_tmp.xs(later_date, level='Date')
# print(quality_df)
print(f"--- Quality Metrics for {later_date} ---")
# print(quality_df.xs('2025-10-03', level='Date'))
print(quality_df.xs(later_date, level='Date'))

In [ ]:
eligible_tickers = get_eligible_universe(quality_metrics_df=quality_df, 
                                        #  filter_date=_start_date, 
                                         filter_date=later_date,                                         
                                         thresholds=ticker_thresholds)

In [ ]:
_df = df_OHLCV.loc[eligible_tickers].copy()
_df.info()

In [ ]:
plot_walk_forward_analyzer_original(
    df_ohlcv=_df,  # CRITICAL: Use the same df_dev as the bot
    default_start_date=_start_date,
    default_calc_period=_calc_period,
    default_fwd_period=_fwd_period,
    default_metric=_metric,
    default_rank_start=_rank_start,
    default_rank_end=_rank_end,
    default_benchmark_ticker=_benchmark_ticker,
)

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load the Bot's Output ---
try:
    results_df = pd.read_csv(bot_config['results_output_path'])
    print(f"Successfully loaded '{bot_config['results_output_path']}'. Shape: {results_df.shape}")
except FileNotFoundError:
    print("Error: The results file was not found. Please run the bot first.")
    # In a real script, you'd exit here. For a notebook, we'll stop.
    results_df = None

if results_df is not None:
    # --- 2. Define the Strategy Parameters for Grouping ---
    # These are the columns that uniquely identify one strategy configuration.
    # strategy_params = ['calc_period', 'metric', 'rank_start', 'rank_end']
    strategy_params = ['calc_period', 'fwd_period', 'metric', 'rank_start', 'rank_end']    

    # --- 3. Group and Aggregate the Results ---
    # We group by the strategy parameters and calculate key performance metrics for each group.
    summary_df = results_df.groupby(strategy_params).agg(
        avg_fwd_p_gain=('fwd_p_gain', 'mean'),
        std_fwd_p_gain=('fwd_p_gain', 'std'),
        avg_fwd_gain_delta=('fwd_gain_delta', 'mean'),
        # Calculate Win Rate: The percentage of periods with positive forward gain.
        win_rate=('fwd_p_gain', lambda x: (x > 0).sum() / len(x) if len(x) > 0 else 0),
        # Count the number of periods tested for this strategy
        num_periods=('step_date', 'count')
    ).sort_values(by='avg_fwd_gain_delta', ascending=False) # Sort by outperformance vs benchmark

    # --- 4. Format and Display the Summary Table ---
    print("\n--- Strategy Performance Summary (2014-2018 Development Run) ---")
    
    # Apply formatting for better readability
    formatted_summary = summary_df.style.format({
        'avg_fwd_p_gain': '{:+.2%}',
        'std_fwd_p_gain': '{:.2%}',
        'avg_fwd_gain_delta': '{:+.2%}',
        'win_rate': '{:.1%}',
    }).set_properties(**{'text-align': 'right'})

    display(formatted_summary)

In [ ]:
import plotly.graph_objects as go

def plot_cumulative_performance(df_ohlcv, strategy_params, quality_thresholds):
    """
    Plots the cumulative performance of a SINGLE strategy over a specified time range.

    This function simulates rebalancing a portfolio at a fixed frequency and charts
    the resulting equity curve against a benchmark.
    """
    print("--- Running Cumulative Performance Simulation for the Winning Strategy ---")
    
    # --- 1. Setup and Pre-processing ---
    # Unpack strategy parameters from the dictionary
    start_date = strategy_params['start_date']
    end_date = strategy_params['end_date']
    calc_period_str = strategy_params['calc_period']
    fwd_period_str = strategy_params['fwd_period']
    metric = strategy_params['metric']
    rank_start = strategy_params['rank_start']
    rank_end = strategy_params['rank_end']
    benchmark_ticker = strategy_params['benchmark_ticker']
    
    # Pre-calculate quality metrics and unstack data once for efficiency
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)

    # Map string periods to pandas DateOffset objects
    period_options = {'3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    calc_period = period_options[calc_period_str]
    fwd_period = period_options[fwd_period_str] # This also defines our rebalancing frequency
    
    # --- 2. Main Simulation Loop ---
    # Create the rebalancing dates
    step_dates = pd.date_range(start=start_date, end=end_date, freq=f'{fwd_period.months}ME')
    
    all_fwd_gains = []
    
    print(f"Simulating from {step_dates[0].date()} to {step_dates[-1].date()}...")
    
    for step_date in step_dates:
        # Get the eligible universe for this specific rebalancing date
        eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
        if not eligible_tickers: continue
            
        df_close_step = df_close_full[eligible_tickers]
        df_high_step = df_high_full[eligible_tickers]
        df_low_step = df_low_full[eligible_tickers]
        
        # Run the core calculation for this single step in time
        step_result = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step,
            step_date, calc_period, fwd_period,
            metric, rank_start, rank_end, benchmark_ticker
        )
        
        if step_result['error'] is None:
            # Extract the forward portion of the portfolio's performance
            fwd_series = step_result['portfolio_series'].loc[step_result['actual_calc_end_ts']:]
            all_fwd_gains.append(fwd_series.pct_change().dropna())
            
    # --- 3. Stitch Together Results & Plot ---
    if not all_fwd_gains:
        print("Error: No valid periods were simulated. Cannot plot.")
        return

    # Concatenate all the forward period returns into one long series
    strategy_returns = pd.concat(all_fwd_gains)
    
    # Create the equity curve (cumulative performance)
    # (1 + returns).cumprod() is the standard way to calculate this
    strategy_equity_curve = (1 + strategy_returns).cumprod()
    
    # Get the benchmark returns for the same period
    benchmark_returns = df_close_full[benchmark_ticker].pct_change()
    benchmark_returns_filtered = benchmark_returns.loc[strategy_equity_curve.index]
    benchmark_equity_curve = (1 + benchmark_returns_filtered).cumprod()
    
    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=strategy_equity_curve.index, y=strategy_equity_curve,
                             mode='lines', name='Winning Strategy', line=dict(color='green', width=3)))
    fig.add_trace(go.Scatter(x=benchmark_equity_curve.index, y=benchmark_equity_curve,
                             mode='lines', name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
    
    fig.update_layout(
        title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end}) vs. Benchmark",
        xaxis_title="Date",
        yaxis_title="Cumulative Growth (Normalized to 1)",
        legend_title="Portfolio",
        hovermode='x unified',
        height=600
    )
    fig.show()

In [ ]:
# --- Define the parameters for our winning strategy ---
winning_strategy_params = {
    'start_date': '2014-01-31',
    'end_date': '2018-12-31',
    'calc_period': '6M',
    'fwd_period': '3M',
    'metric': 'Sharpe (ATR)',
    'rank_start': 1,
    'rank_end': 10,
    'benchmark_ticker': 'VOO'
}

# Get the quality thresholds from the bot's configuration
quality_thresholds_from_bot = bot_config['quality_thresholds']

# --- Run the simulation and generate the plot ---
plot_cumulative_performance(
    df_ohlcv=df_dev,  # Run on our development dataset
    strategy_params=winning_strategy_params,
    quality_thresholds=quality_thresholds_from_bot
)